<a href="https://colab.research.google.com/github/teyang-lau/you-only-edit-once/blob/main/notebooks/postprocessing/Moviemaker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Welcome to a demonstration of the moviemaker
This demonstration is done on colab.

Install libraries

In [4]:
!pip install -q pafy
!pip install -q youtube-dl
!pip install -q ffmpeg-python
!apt install ffmpeg
!pip install -q pytube
!pip install -q librosa
!pip install -q pydub
!pip install -q soundfile

     |████████████████████████████████| 1.9 MB 5.1 MB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.11-0ubuntu0.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 12 not upgraded.
     |████████████████████████████████| 56 kB 2.4 MB/s 


In [6]:
from __future__ import unicode_literals

import os
from datetime import datetime

import cv2
import pafy
import youtube_dl
import librosa
import soundfile as sf
import ffmpeg

from google.colab import files
from pydub import AudioSegment

Upload video file - only if you're using Colab

In [7]:
uploaded = files.upload()

Saving fish_video.mp4 to fish_video.mp4


In [8]:
video_file = list(uploaded.keys())[0]

Let's make a video

In [9]:
youtube_link = 'https://www.youtube.com/watch?v=VIkxcLNjBnM'   # @param {type:"string"}

In [10]:
output_path = f'movie_{datetime.now().strftime("%Y%m%d-%H%M%S")}.mp4'

In [13]:
def moviemaker(audio_file, video_file, output_path, youtube_url=False):
  '''
  Creates a movie depending with the duration equivalent to the shortest duration
  between audio and video files.

  audio_file: Path to the audio file. This is required if youtube_url = False. 
  If youtube_url = True, this can be set to None
  video_file: Path to the .mp4 file. This is required.
  output_path: This is required.
  youtube_url: The default is set to False
  '''
  #Download audio from YouTube
  if youtube_url:
    audio_file = './downloaded_audio.wav'

    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': audio_file,
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'wav',
            'preferredquality': '192'
        }],
        'postprocessor_args': [
            '-ar', '16000'
        ],
        'prefer_ffmpeg': True,
        'keepvideo': False
    }

    with youtube_dl.YoutubeDL(ydl_opts) as ydl:
        ydl.download([youtube_url])
      
    print('Audio downloaded!')
  
  #Assemble video
  sound = AudioSegment.from_file(audio_file)
  audio_duration = librosa.get_duration(filename = audio_file)

  vid = cv2.VideoCapture(video_file)
  frames = vid.get(cv2.CAP_PROP_FRAME_COUNT)
  fps = vid.get(cv2.CAP_PROP_FPS)
  video_duration = frames/fps

  if audio_duration > video_duration:
    audio_file = './audio_truncated.wav'
    first_cut_point = (0) * 1000 #miliseconds
    last_cut_point = round(video_duration) * 1000
    sound_clip = sound[first_cut_point:last_cut_point]
    sound_clip.export(audio_file, format="wav")
  
  else:
    clone = video_duration//audio_duration +1 
    audio_file = './audio_extended.wav'
    sound*=clone #the audio file is now longer than the video file
    first_cut_point = (0)*1000
    last_cut_point = round(video_duration)*1000
    sound_clip = sound[first_cut_point:last_cut_point]
    sound_clip.export(audio_file, format='wav')

  input_video = ffmpeg.input(video_file)
  input_audio = ffmpeg.input(audio_file)
  ffmpeg.concat(input_video, input_audio, v=1, a=1).output(output_path).run()
  
  print('Movie made! Congratulations!')

In [14]:
moviemaker(audio_file = None,
           video_file=video_file,
           output_path = output_path,
           youtube_url = youtube_link)

files.download(output_path)
print('\nEnjoy your movie!')

[youtube] VIkxcLNjBnM: Downloading webpage
[download] Destination: ./downloaded_audio.wav
[download] 100% of 4.05MiB in 01:01
[ffmpeg] Post-process file ./downloaded_audio.wav exists, skipping
Audio downloaded!
Movie made! Congratulations!


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Enjoy your movie!
